# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [59]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [61]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x
% pip install tensorflow-gpu==2.1

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [62]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics, callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from jarvis.train import datasets, custom
from jarvis.train.client import Client
from jarvis.utils.general import overload, tools as jtools
from jarvis.utils.display import imshow
import os

# Data

The data used in this tutorial will consist of (frontal projection) chest radiographs from the RSNA / Kaggle pneumonia challenge (https://www.kaggle.com/c/rsna-pneumonia-detection-challenge). The chest radiograph is the standard screening exam of choice to identify and trend changes in lung disease including infection (pneumonia). To simulate the problem of small dataset size, only 100 exams will be used for training (50 normal, 50 positive). A separate 100 exams will be used for independent testing.

The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/xr_pna`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [64]:
# --- Download dataset
datasets.download(name='xr/pna-mul')

{'code': '/data/raw/xr_pna', 'data': '/data/raw/xr_pna'}

# Training

To account for the small training cohort size, three separate strategies must be employed:

1. A **dual loss function** network with both classification and segmentation losses (as demonstrated in the tutorial)

2. A reduced final feature map of either (`8 x 8`) or (`4 x 4`) instead of the (`16 x 16`) used in the demonstration

3. At least one of the following (or both) regularizer techniques:

* dropout
* L2 regularization

Feel free to further optimize the network architecture to further improve model performance. Recommendings include limiting model size to reduce overfitting, early stopping and mixed L1/L2 regularization.

### Overload the `Client` object

*Hint*: Ensure to customize the `arrays['xs']['msk']` object to reflect masked values as shown in the tutorial.

In [0]:
@overload(Client)

def preprocess(self, arrays,**kwargs):

  msk = np.zeros(arrays['xs']['dat'].shape)

  lng = arrays['xs']['msk']
  pna = arrays['ys']['pna-seg']

  msk[lng > 0] = 1
  msk[pna > 0] = 3

  arrays['xs']['msk'] = msk

  return arrays

### Create `Client` object

After manually overloading the `Client` object, manually create a new client object.

In [0]:
# --- Find client yml file
yml = '{}/data/ymls/client-mul.yml'.format(jtools.get_paths('xr/pna')['code'])

# --- Manually create Client
client = Client(yml)


### Create inputs and generators

In [0]:
# --- Manually create generators
gen_train, gen_valid = client.create_generators()
inputs = client.get_inputs(Input)

### Define the model

*Hint*: Ensure to use both a classification and segmentation (contracting-encoding) architecture as demonstrated in the tutorial. At this point, also ensure to use either dropout or L2 regularization; other regularization techniques may also be explored to improve model performance.

In [0]:
kwargs = {
    'kernel_size': (1,3,3),
    'padding': 'same'}


conv = lambda x, filters, strides, kernel_regularizer : layers.Conv3D(filters = filters, strides = strides, kernel_regularizer = kernel_regularizer, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
leakyrelu = lambda x: layers.LeakyReLU()(x)
transpose = lambda x, filters, strides, kernel_regularizer : layers.Conv3DTranspose(filters = filters, strides = strides, kernel_regularizer = kernel_regularizer, **kwargs)(x)


conv1 = lambda filters, x, : leakyrelu(norm(conv(x, filters = filters, strides = (1,1,1), kernel_regularizer = None)))
conv2 = lambda filters, x, : leakyrelu(norm(conv(x, filters = filters, strides = (1,2,2), kernel_regularizer = None)))
tran = lambda filters, x : leakyrelu(norm(transpose(x, filters, strides = (1,2,2), kernel_regularizer = None)))


In [124]:
dropout_rate = 0.1

c1 = conv1(8, inputs['dat'])
c2 = conv1(16, conv2(16,c1))
c3 = conv1(32, conv2(32,c2))
c3_d = layers.Dropout(rate = dropout_rate)(c3)
c4 = conv1(48, conv2(48,c3_d))
c4_d = layers.Dropout(rate = dropout_rate)(c4)
c5 = conv1(64, conv2(64,c4_d))
c5_d = layers.Dropout(rate = dropout_rate)(c5)
c6 = conv1(96, conv2(96,c5_d))
c6_d = layers.Dropout(rate = dropout_rate)(c6)
c7 = conv1(96, conv2(96,c6_d))
c7_d = layers.Dropout(rate = dropout_rate)(c7)

c8 = conv1(96, conv2(96,c7_d))
c8_d = layers.Dropout(rate = dropout_rate)(c8)

c9 = conv1(96, conv2(96,c8_d))
c9_d = layers.Dropout(rate = dropout_rate)(c9)


r1 = layers.Reshape((-1,1,1,2*2*96))(c9_d)


e1 = tran(96,c8_d)
e1_d = layers.Dropout(rate = dropout_rate)(e1)
e2 = tran(96, conv1(96, e1_d + c7_d))
e2_d = layers.Dropout(rate = dropout_rate)(e2)
e3 = tran(64, conv1(96, e2_d + c6_d))
e3_d = layers.Dropout(rate = dropout_rate)(e3)
e4 = tran(48, conv1(64, e3_d + c5_d))
e4_d = layers.Dropout(rate = dropout_rate)(e4)
e5 = tran(32, conv1(48, e4_d + c4_d))
e5_d = layers.Dropout(rate = dropout_rate)(e5)
e6 = tran(16, conv1(32, e5_d + c3_d))
e7 = tran(8, conv1(16, e6+c2))
e8 = conv1(8, e7+c1)




logits = {}

logits['pna-seg'] = layers.Conv3D(filters = 2, name = 'pna-seg', **kwargs)(e8)
logits['pna-cls'] = layers.Conv3D(2, name = 'pna-cls', kernel_size = (1,1,1))(r1)


model = Model(inputs = inputs, outputs = logits)

model.summary()

Model: "model_20"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, 1, 512, 512, 0                                            
__________________________________________________________________________________________________
conv3d_446 (Conv3D)             (None, 1, 512, 512,  80          dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization_586 (BatchN (None, 1, 512, 512,  32          conv3d_446[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_586 (LeakyReLU)     (None, 1, 512, 512,  0           batch_normalization_586[0][0]    
___________________________________________________________________________________________

### Compile the model

*Hint*: Ensure that custom loss functions are used as described in the tutorial to properly mask the segmentation loss function.

In [0]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'pna-seg': custom.sce(inputs['msk']),
        'pna-cls': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={
        'pna-seg': custom.dsc(weights=inputs['msk']),
        'pna-cls': metrics.SparseCategoricalAccuracy()
        },
    experimental_run_tf_function=False)

### Train the model

Use the following cell block to train your model.

In [0]:
fname1 = '{}/models/disease_characterization/model_checkpoint.h5'.format(MOUNT_ROOT)
check_point = ModelCheckpoint(fname1,monitor='val_pna-cls_sparse_categorical_accuracy',verbose=True, save_best_only=True, save_weights_only=True, mode="max", save_freq="epoch")
reduce_lr = ReduceLROnPlateau(monitor='val_pna-cls_sparse_categorical_accuracy', factor=0.2, patience=2, mode = "max")

In [127]:
history = model.fit(
    x = gen_train,
    steps_per_epoch = 50,
    epochs = 20,
    validation_data = gen_valid,
    validation_steps = 100,
    validation_freq = 1,
    callbacks = [check_point, reduce_lr])

Epoch 1/20
100/50 [============================================================] - 13s 127ms/step - loss: 0.9298 - pna-cls_loss: 0.6916 - pna-seg_loss: 0.2403 - pna-cls_sparse_categorical_accuracy: 0.5325 - pna-seg_dsc_1: 0.3579

Epoch 00001: val_pna-cls_sparse_categorical_accuracy improved from -inf to 0.53250, saving model to /content/drive/My Drive/models/disease_characterization/model_checkpoint.h5
50/50 [==============================] - 35s 706ms/step - loss: 0.6317 - pna-cls_loss: 0.4123 - pna-seg_loss: 0.2195 - pna-cls_sparse_categorical_accuracy: 0.7975 - pna-seg_dsc_1: 0.2667 - val_loss: 0.9319 - val_pna-cls_loss: 0.6916 - val_pna-seg_loss: 0.2403 - val_pna-cls_sparse_categorical_accuracy: 0.5325 - val_pna-seg_dsc_1: 0.3579
Epoch 2/20
100/50 [============================================================] - 11s 109ms/step - loss: 0.9757 - pna-cls_loss: 0.7011 - pna-seg_loss: 0.2225 - pna-cls_sparse_categorical_accuracy: 0.4875 - pna-seg_dsc_1: 0.3270

Epoch 00002: val_pna-cls_s

In [0]:
model.load_weights(fname1)

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. Only model accuracy (global classification performance) needs to evaluated (Dice score does not).

### Performance

The following minimum performance metrics must be met for full credit:

* accuracy: >0.82

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort accuracy statistics. There is no need to submit training performance accuracy.

In [129]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

accuracy = []

for x, y in test_valid:
    
    # --- Create prediction
    logits = model.predict(x)
    pred = np.argmax(logits[0], axis=-1)
    
    # --- Compare with ground truth
    accuracy.append(pred.squeeze() == y['pna-cls'].squeeze())

accuracy = np.array(accuracy)

print(accuracy.mean())
df = []
df = pd.DataFrame(index=np.arange(accuracy.size))
df['accuracy'] = accuracy

[ 2020-05-28 21:28:57 ] [====================] 100.000% : Iterating | 000100    0.94
